In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Get the beta Estimates of the stocks

In [2]:
# Get the exceess returns data
returndata = pd.read_csv('data/returndata_benchmark_excess_ret.csv', index_col=0)

In [3]:
returndata.head()

,date,permno,ret,mktcap,rf,excess_ret,benchmark_excess_ret
0,192601,10006,0.032732,66150.00,NaN,NaN,0.0
1,192601,10022,0.017857,11400.00,NaN,NaN,0.0
2,192601,10030,0.161667,27183.00,NaN,NaN,0.0
3,192601,10049,0.141892,20875.00,NaN,NaN,0.0
4,192601,10057,-0.035714,5906.25,NaN,NaN,0.0


In [4]:
def get_beta(span, stock_return_data):
    return stock_return_data.excess_ret.ewm(span=span).cov(stock_return_data.benchmark_excess_ret)/stock_return_data.benchmark_excess_ret.ewm(span=span).var()

In [5]:
# For beta esstimate, we wish to take last 5 years of data and get the EWM weighted Beta values
spans = [5, 10, 20, 30]

In [6]:
for span in spans:
    betas = returndata.groupby('permno').apply(lambda x: get_beta(span=span, stock_return_data=x.set_index('date'))).reset_index().rename(columns={0: f'beta_{span}'})
    returndata = pd.merge(returndata, betas, on=['date', 'permno'], how='left')

In [7]:
returndata[returndata.date > 200001].head()

,date,permno,ret,mktcap,rf,excess_ret,benchmark_excess_ret,beta_5,beta_10,beta_20,beta_30
2759047,200002,10001,0.015385,20212.50000,0.43,-0.414615,-0.348927,0.548127,0.567399,0.512553,0.523175
2759048,200002,10002,-0.163158,76806.93750,0.43,-0.593158,-0.348927,-0.625304,-0.323440,0.034102,0.198341
2759049,200002,10009,-0.028120,32514.28125,0.43,-0.458120,-0.348927,-0.393069,-0.002453,0.310357,0.426010
2759050,200002,10012,0.133621,159032.81250,0.43,-0.296379,-0.348927,1.472567,1.159767,1.875727,2.068863
2759051,200002,10016,0.132353,343978.25000,0.43,-0.297647,-0.348927,0.792324,0.739925,0.895522,0.958790


In [8]:
returndata.to_csv('data/return_pf_excess_ret_beta.csv')